# webpack 初始化

## createCompiler

webpack 初始化的任务

1. 初始化日志系统，确定文件系统[输入系统，输出系统]，输出系统又包括内存文件系统，和直接的文件系统
2. 创立 compiler [MultiCompiler，Compiler]  其中 MultiCompiler 针对 数组 options
3. 执行 compiler

In [ ]:
// 不管是MultCompiler 还是 Compiler 最终都会调用 createCompiler
const createCompiler = rawOptions => {
	const options = getNormalizedWebpackOptions(rawOptions);
	// options.context = cwd
	// 日志系统
	applyWebpackOptionsBaseDefaults(options);
	const compiler = new Compiler(options.context, options);
	// 日志，文件系统
	new NodeEnvironmentPlugin({
		infrastructureLogging: options.infrastructureLogging
	}).apply(compiler);
	// 插件调用
	if (Array.isArray(options.plugins)) {
		for (const plugin of options.plugins) {
			if (typeof plugin === "function") {
				plugin.call(compiler, compiler);
			} else {
				plugin.apply(compiler);
			}
		}
	}
	// 应用 webpack config
	applyWebpackOptionsDefaults(options);
	compiler.hooks.environment.call();
	compiler.hooks.afterEnvironment.call();
	new WebpackOptionsApply().process(options, compiler);
	compiler.hooks.initialize.call();
	return compiler;
};

In [ ]:
// create 函数就是createCompiler的过程
const { compiler, watch, watchOptions } = create();
// 监听变化
if (watch) {
    compiler.watch(watchOptions, callback);
// 生成文件
} else {
    compiler.run((err, stats) => {
        compiler.close(err2 => {
            callback(err || err2, stats);
        });
    });
}
return compiler;